# TPG Examples
This document shows how to use the PyTPG API. We make use of OpenAI Gym to run examples, and we assume you already have PyTPG installed, see the readme for installation instructions for PyTPG.

## General Setup

In [1]:
from __future__ import division
from IPython.display import clear_output
import numpy as np
import gym
import matplotlib.pyplot as plt
from IPython import display

# how to render in Jupyter: 
# https://stackoverflow.com/questions/40195740/how-to-run-openai-gym-render-over-a-server
# https://www.youtube.com/watch?v=O84KgRt6AJI
def show_state(env, step=0, name='', info=''):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (name, step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())
    
# To transform pixel matrix to a single vector.
def getState(inState):
    # each row is all 1 color
    rgbRows = np.reshape(inState,(len(inState[0])*len(inState), 3)).T

    # add each with appropriate shifting
    # get RRRRRRRR GGGGGGGG BBBBBBBB
    return np.add(np.left_shift(rgbRows[0], 16),
        np.add(np.left_shift(rgbRows[1], 8), rgbRows[2]))

In [2]:
env = gym.make('Assault-v0') # make the environment

In [3]:
print(env.action_space) # learn size of action space

Discrete(7)


In [4]:
# import to do training
from tpg.trainer import Trainer
# import to run an agent (always needed)
from tpg.agent import Agent

## Example: Basic Generational Selection (with graphics)

In [ ]:
%matplotlib inline
# takes too long with rendering so I just ignore this one, but it works

import time # for tracking time

tStart = time.time()

# first create an instance of the TpgTrainer
# this creates the whole population and everything
# teamPopSize should realistically be at-least 100
trainer = Trainer(actions=range(7), teamPopSize=20, rTeamPopSize=20) 

curScores = [] # hold scores in a generation
summaryScores = [] # record score summaries for each gen (min, max, avg)

# 5 generations isn't much (not even close), but some improvements
# should be seen.
for gen in range(5): # generation loop
    curScores = [] # new list per gen
    
    agents = trainer.getAgents()
    
    while True: # loop to go through agents
        teamNum = len(agents)
        agent = agents.pop()
        if agent is None:
            break # no more agents, so proceed to next gen
        
        state = env.reset() # get initial state and prep environment
        score = 0
        for i in range(500): # run episodes that last 500 frames
            show_state(env, i, 'Assault', 'Gen #' + str(gen) + 
                       ', Team #' + str(teamNum) +
                       ', Score: ' + str(score)) # render env
            
            # get action from agent
            # must transform to at-least int-32 (for my getState to bitshift correctly)
            act = agent.act(getState(np.array(state, dtype=np.int32))) 

            # feedback from env
            state, reward, isDone, debug = env.step(act)
            score += reward # accumulate reward in score
            if isDone:
                break # end early if losing state

        agent.reward(score) # must reward agent (if didn't already score)
            
        curScores.append(score) # store score
        
        if len(agents) == 0:
            break
            
    # at end of generation, make summary of scores
    summaryScores.append((min(curScores), max(curScores),
                    sum(curScores)/len(curScores))) # min, max, avg
    trainer.evolve()
    
#clear_output(wait=True)
print('Time Taken (Hours): ' + str((time.time() - tStart)/3600))
print('Results:\nMin, Max, Avg')
for result in summaryScores:
    print(result[0],result[1],result[2])

## Example: Generational Selection with Multiprocessing (no graphics)

This example is just to show a different way to run the API, a far superior way. It uses a different method to get the agents, doesn't use graphics (but can), and uses multiprocessing.

In [5]:
"""
Run each agent in this method for parallization.
Args:
    args: (TpgAgent, envName, scoreList, numEpisodes, numFrames)
"""
def runAgent(args):
    agent = args[0]
    envName = args[1]
    scoreList = args[2]
    numEpisodes = args[3] # number of times to repeat game
    numFrames = args[4] 
    
    # skip if task already done by agent
    if agent.taskDone(envName):
        print('Agent #' + str(agent.agentNum) + ' can skip.')
        scoreList.append((agent.team.id, agent.team.outcomes))
        return
    
    env = gym.make(envName)
    valActs = range(env.action_space.n) # valid actions, some envs are less
    
    scoreTotal = 0 # score accumulates over all episodes
    for ep in range(numEpisodes): # episode loop
        state = env.reset()
        scoreEp = 0
        numRandFrames = 0
        if numEpisodes > 1:
            numRandFrames = random.randint(0,30)
        for i in range(numFrames): # frame loop
            if i < numRandFrames:
                _, _, isDone, _ = env.step(env.action_space.sample())
                if isDone: # don't count it if lose on random steps
                    ep -= 1
                continue

            act = agent.act(getState(state))

            # feedback from env
            state, reward, isDone, debug = env.step(act)
            scoreEp += reward # accumulate reward in score
            if isDone:
                break # end early if losing state
                
        print('Agent #' + str(agent.agentNum) + 
              ' | Ep #' + str(ep) + ' | Score: ' + str(scoreEp))
        scoreTotal += scoreEp
       
    scoreTotal /= numEpisodes
    env.close()
    agent.reward(scoreTotal, envName)
    scoreList.append((agent.team.id, agent.team.outcomes))

In [6]:
import multiprocessing as mp
import time

tStart = time.time()

envName = 'Boxing-v0'
# get num actions
env = gym.make(envName)
acts = env.action_space.n
del env

trainer = Trainer(actions=range(acts), teamPopSize=200, rTeamPopSize=200)

processes = 7
man = mp.Manager()
pool = mp.Pool(processes=processes, maxtasksperchild=1)
    
allScores = [] # track all scores each generation

for gen in range(100): # do 100 generations of training
    scoreList = man.list()
    
    # get agents, noRef to not hold reference to trainer in each one
    # don't need reference to trainer in multiprocessing
    agents = trainer.getAgents() # swap out agents only at start of generation

    # run the agents
    pool.map(runAgent, 
        [(agent, envName, scoreList, 1, 18000)
        for agent in agents])
    
    # apply scores, must do this when multiprocessing
    # because agents can't refer to trainer
    teams = trainer.applyScores(scoreList)
    # important to remember to set tasks right, unless not using task names
    # task name set in runAgent()
    trainer.evolve(task=envName) # go into next gen
    
    # an easier way to track stats than the above example
    scoreStats = trainer.fitnessStats
    allScores.append((scoreStats['min'], scoreStats['max'], scoreStats['average']))
    
    clear_output()
    print('Time Taken (Hours): ' + str((time.time() - tStart)/3600))
    print('Results so far: ' + str(allScores))
    
clear_output()
print('Time Taken (Hours): ' + str((time.time() - tStart)/3600))
print('Results:\nMin, Max, Avg')
for score in allScores:
    print(score[0],score[1],score[2])

Time Taken (Hours): 2.909954893456565
Results:
Min, Max, Avg
-100.0 -2.0 -46.915
-100.0 2.0 -34.775
-100.0 2.0 -36.17412935323383
-100.0 2.0 -29.800995024875622
-100.0 2.0 -27.437810945273633
-100.0 4.0 -25.686567164179106
-100.0 4.0 -24.203980099502488
-100.0 4.0 -24.228855721393035
-100.0 4.0 -19.104477611940297
-100.0 4.0 -19.98
-100.0 5.0 -17.39
-100.0 8.0 -16.895
-100.0 8.0 -16.885
-100.0 10.0 -14.681592039800995
-100.0 12.0 -11.88
-100.0 13.0 -12.55
-100.0 13.0 -16.38
-100.0 19.0 -13.39
-100.0 19.0 -13.119402985074627
-100.0 19.0 -10.125
-100.0 19.0 -12.09
-100.0 19.0 -9.325
-100.0 19.0 -9.185
-100.0 19.0 -10.635
-100.0 22.0 -7.52
-100.0 22.0 -10.485
-100.0 22.0 -10.425
-100.0 22.0 -10.368159203980099
-100.0 22.0 -9.9
-100.0 22.0 -8.58
-100.0 22.0 -10.45273631840796
-100.0 22.0 -9.73
-100.0 22.0 -8.405
-100.0 30.0 -11.64
-100.0 30.0 -9.2
-100.0 30.0 -11.49
-100.0 30.0 -8.475
-100.0 30.0 -10.59
-100.0 30.0 -8.83
-100.0 30.0 -9.805
-100.0 30.0 -7.517412935323383
-100.0 30.0 -8.7
-1